In [1]:
import os


In [2]:
from openai import OpenAI

client = OpenAI()

In [3]:
def get_response(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content



In [ ]:
response = get_response("안녕! 넌 누구야?")

response

In [5]:
from llama_index.llms.openai import OpenAI

from llama_index.core.llms import ChatMessage

client_llama = OpenAI(model="gpt-3.5-turbo")

In [6]:
message = [ChatMessage(role='user', content='안녕! 넌 누구야?')]

In [ ]:
response = client_llama.chat(message)

response

In [ ]:
response.message.content

In [ ]:
query = "OpenAI의 sora 모델에 대해서 설명해줘"

resp = get_response(query)

print(resp)

In [ ]:
prompt = f"""
Utilizing the given context, please answer the question.

[context]
Sora

Main Article: Sora (text-to-video-model)
Sora is a text-to-video model that can generate videos based on short descriptive prompts[236] as well as extend existing videos forwards or backwards in time.[237] It can generate videos with resolution up to 1920x1080 or 1080x1920. The maximal length of generated videos is unknown.

Sora's development team named it after the Japanese word for "sky", to signify its "limitless creative potential".[236] Sora's technology is an adaptation of the technology behind the DALL·E 3 text-to-image model.[238] OpenAI trained the system using publicly-available videos as well as copyrighted videos licensed for that purpose, but did not reveal the number or the exact sources of the videos.[236]

OpenAI demonstrated some Sora-created high-definition videos to the public on February 15, 2024, stating that it could generate videos up to one minute long. It also shared a technical report highlighting the methods used to train the model, and the model's capabilities.[238] It acknowledged some of its shortcomings, including struggles simulating complex physics.[239] Will Douglas Heaven of the MIT Technology Review called the demonstration videos "impressive", but noted that they must have been cherry-picked and might not represent Sora's typical output.[238]

[user question]

{query}
"""

resp = get_response(prompt)

print(resp)


In [ ]:
prompt = f"""
Utilizing the given context, please answer the question.

[context]
On March 14, 2023, OpenAI announced the release of Generative Pre-trained Transformer 4 (GPT-4), capable of accepting text or image inputs.[212] They announced that the updated technology passed a simulated law school bar exam with a score around the top 10% of test takers. (By contrast, GPT-3.5 scored around the bottom 10%.) They said that GPT-4 could also read, analyze or generate up to 25,000 words of text, and write code in all major programming languages.[213]

Observers reported that the iteration of ChatGPT using GPT-4 was an improvement on the previous GPT-3.5-based iteration, with the caveat that GPT-4 retained some of the problems with earlier revisions.[214] GPT-4 is also capable of taking images as input on ChatGPT.[215] OpenAI has declined to reveal various technical details and statistics about GPT-4, such as the precise size of the model.[216]
[user question]

{query}
"""

resp = get_response(prompt)

print(resp)



# Load data

In [ ]:
import requests
response = requests.get(
  "https://en.wikipedia.org/w/api.php",
  params={
    "action": "query",
    "format": "json",
    "titles": "OpenAI",
    "prop": "extracts",
    "explaintext": True,
  },
).json()

response

In [ ]:
page = next(iter(response['query']['pages'].values()))

text = page['extract']
text

# Indexing

In [14]:
def get_chunks(text, chunk_size):
  words = text.split()
  chunks = [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
  return chunks

In [ ]:
chunks = get_chunks(text, 128)
chunks

In [ ]:
client.embeddings.create(input=['머신러닝'], model='text-embedding-3-small')

In [17]:
def get_embedding(text):
  return client.embeddings.create(input=text, model='text-embedding-3-small').data[0].embedding

In [ ]:

embeddings = [get_embedding(c) for c in chunks]

embeddings


# retrieve

In [34]:
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

def retriever(query, top_k):
  q_emb = get_embedding(query)

  sim_score = cosine_similarity([q_emb], embeddings)[0]

  max_indices = np.argsort(sim_score)[::-1][:top_k]

  retrieved_datas = [chunks[i] for i in max_indices]

  return retrieved_datas

In [ ]:
contexts = retriever("OpenAI의 sora 모델에 대해 설명해줘 ", 3)
contexts

# Generate

In [37]:
def generator(query, contexts):
  context = "\n\n".join(contexts)

  prompt = f"""

  Utilizing the given context, please answer the question.
  [context]
  {context}

  [user question]
  {query}
  """

  return get_response(prompt)

In [ ]:
generator("OpenAI의 sora 모델에 대해 알려줘", contexts)

# LlamaIndex

In [40]:
from pathlib import Path

data_path = Path("../dataset/llamaindex_data")

if not data_path.exists():
  Path.mkdir(data_path)

with open(f"{data_path}/openai.txt", "w") as fp:
  fp.write(text)


# Load data



In [46]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("../dataset/llamaindex_data").load_data()
vector_index = VectorStoreIndex.from_documents(documents)

In [ ]:
%pip install llama-index-embeddings-openai

In [47]:
query_engine = vector_index.as_query_engine()

In [ ]:
response = query_engine.query(query)

print(response)

In [ ]:
from llama_index.core import Settings

Settings

In [ ]:
Settings._llm

In [ ]:
Settings._embed_model

In [ ]:
Settings._node_parser

# Chunk Size 조절하기

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=128, chunk_overlap=10)

nodes = text_splitter.get_nodes_from_documents(documents)

len(nodes)

In [ ]:
from llama_index.core.node_parser import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=10, include_metadata=False)

nodes = text_splitter.get_nodes_from_documents(documents)

len(nodes)



In [55]:
from llama_index.core.schema import TextNode

node_new = TextNode(text = "OpenAI의 소라는 text to video 모델로, 생성할 수 있는 비디오는 오직 투디 애니메이션이다. 대표적인 창작물로는 NLP에 대한 소개 애니메이션이 있다.")

nodes.append(node_new)


In [56]:
vector_index = VectorStoreIndex(nodes)

In [ ]:
query_engine = vector_index.as_query_engine()
response = query_engine.query(query)

response.response

In [ ]:
get_response("Mistral AI는 어느 국적의 회사야?")

In [59]:
documents = SimpleDirectoryReader("../dataset/llamaindex_data").load_data()

vector_index = VectorStoreIndex.from_documents(documents)

query_engine = vector_index.as_query_engine()

In [ ]:
query_engine.query("Mistral AI는 어느 국적의 회사야?").response

In [ ]:
query_engine.query("Mistral AI는 어느 국적의 회사야?")

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(model="text-embedding-3-small", dimension=128)

embed_model.get_text_embedding("머신러닝은 멋져")

In [64]:
Settings.embed_model = embed_model

In [65]:
from llama_index.core.retrievers import VectorIndexRetriever

retriever = VectorIndexRetriever(
  index=vector_index, 
  similarity_top_k=3
)

In [66]:
nodes = retriever.retrieve("Mistral AI는 어느 국적의 회사야?")

In [ ]:
nodes

In [ ]:
from llama_index.core import DocumentSummaryIndex

chatgpt = OpenAI(model="gpt-3.5-turbo")

doc_summary_index = DocumentSummaryIndex.from_documents(
  documents, 
  llm=chatgpt, 
  show_progress=True
)


In [ ]:
doc_summary_index.get_document_summary(documents[0].id_)

In [ ]:
from llama_index.core.indices.document_summary import DocumentSummaryIndexLLMRetriever

retriever = DocumentSummaryIndexLLMRetriever(
  index=doc_summary_index,
  choice_batch_size=3,
)

retriever.retrieve("Mistral AI는 어느 국가에 속해 있어?")


In [ ]:
from llama_index.core.indices.document_summary import DocumentSummaryIndexEmbeddingRetriever

retriever = DocumentSummaryIndexEmbeddingRetriever(
  index=doc_summary_index,
  similarity_top_k=3,
)

retriever.retrieve("Mistral AI는 어느 국가에 속해 있어?")


# Generator

In [74]:
from llama_index.core import get_response_synthesizer

response_synthesizer = get_response_synthesizer(
  response_mode="compact",
)


In [ ]:
query = "Mistral AI는 어느 국가에 속해 있어?"

response = response_synthesizer.synthesize(query, nodes=retriever.retrieve(query))

response.response

In [ ]:
response_synthesizer = get_response_synthesizer(response_mode="accumulate")

response = response_synthesizer.synthesize(query, nodes=retriever.retrieve(query))
response.response

In [ ]:
response_synthesizer = get_response_synthesizer(response_mode="simple_summarize")

response = response_synthesizer.synthesize(query, nodes=retriever.retrieve(query))
response.response

In [ ]:
response_synthesizer = get_response_synthesizer(response_mode="tree_summarize")

response = response_synthesizer.synthesize(query, nodes=retriever.retrieve(query))
response.response

In [ ]:
response_synthesizer.get_prompts()

In [ ]:
prompt_dict = query_engine.get_prompts()
prompt_dict

In [ ]:
for prompt in prompt_dict.values():
  print(prompt.get_template())
  print("************\n\n")

In [82]:
from llama_index.core import PromptTemplate

new_template_qa = """
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Use proper korean, answer in the style of kindergarten teacher, gentle and enthusiastic.
Query: {query_str}
Answer:
"""

new_prompt_refine = """
The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Use proper korean, answer in the style of kindergarten teacher, gentle and enthusiastic.
Refined Answer: 
"""



In [83]:
new_template_qa = PromptTemplate(new_template_qa)
new_prompt_refine = PromptTemplate(new_prompt_refine)

In [85]:
query_engine.update_prompts(
  {"response_synthesizer:text_qa_template": new_template_qa,
   "response_synthesizer:refine_template": new_prompt_refine}
)

In [ ]:
query_engine.get_prompts()

In [ ]:
response = query_engine.query(query)

response.response